In [1]:
# Set up the mount of the google drive.
!pip install librosa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kd_tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packag

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

Using TensorFlow backend.


In [0]:
# Load wav data.
import pickle
filename = '/content/drive/My Drive/CSCE 666/Project/Feature/w_c_dataset.pickle'
infile = open(filename,'rb')
[sound, sr, lengths, times, wc_label] = pickle.load(infile)
infile.close()

In [0]:
# Obtain wav file ID.
import glob
wav_list = [f.split('.wav')[0] for f in glob.glob("/content/drive/My Drive/CSCE 666/Project/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/*.wav")]
wav_list.sort()

wav_ids = []
wav_names = []
for wav_path in wav_list:
  ind = wav_path.rfind("/")
  wav_name = wav_path[ind+1:]
  Id = int(wav_name[:wav_name.find("_")])
  wav_ids.append(Id)
  wav_names.append(wav_name)

In [0]:
import pandas as pd
demographic = pd.read_csv('/content/drive/My Drive/CSCE 666/Project/Feature/wav_info_COPD_missdata_samplemean.csv', header=None)
demographic_data = demographic.values
patient_disease_labels = demographic_data[:, -1]
demographic_data = demographic_data[0:, 1:-1] # Remove the id column (the first column)

In [0]:
"""
# 8 disease classification
def updatelabel(labels): 
    l=[]
    for label in labels:
      if label == 0:
        a = np.array([1, 0, 0, 0, 0, 0, 0, 0])
      elif label == 1:
        a = np.array([0, 1, 0, 0, 0, 0, 0, 0])
      elif label == 2:
        a = np.array([0, 0, 1, 0, 0, 0, 0, 0])
      elif label == 3:
        a = np.array([0, 0, 0, 1, 0, 0, 0, 0])  
      elif label == 4:
        a = np.array([0, 0, 0, 0, 1, 0, 0, 0])  
      elif label == 5:
        a = np.array([0, 0, 0, 0, 0, 1, 0, 0])
      elif label == 6:
        a = np.array([0, 0, 0, 0, 0, 0, 1, 0])
      else:
        a = np.array([0, 0, 0, 0, 0, 0, 0, 1])
      l.append(a)
    return l
  """
# COPD vs. non-COPD
def updatelabel(labels):
    l=[]
    for label in labels:
      if label == 0:
        a = np.array([1, 0])
      if label == 1:
        a = np.array([0, 1]) 
      l.append(a)
    return l

In [9]:
new_patient_disease_labels = updatelabel(patient_disease_labels)
print(new_patient_disease_labels)

[array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0,

In [10]:
print(type(demographic_data))
print(demographic_data.shape)
print(demographic_data[0])
print(demographic_data[1])
print(demographic_data[2])
print(demographic_data[3])
print(demographic_data[4])
print(demographic_data[0].shape)
#print(demographic_data[0:, 1:])
print(len(sound))

<class 'numpy.ndarray'>
(920, 5)
[ 3.    0.   27.19 19.   99.  ]
[ 3.    0.   27.19 19.   99.  ]
[ 0.75  0.   27.19  9.8  73.  ]
[ 70.      0.     33.     21.361 104.65 ]
[ 70.      0.     28.47   21.361 104.65 ]
(5,)
920


In [0]:
import librosa
def find_mfcc_max_length(sound, sr):
  num_sound = len(sound)
  max_length = 0
  for i in range(num_sound):
    mfccs = librosa.feature.mfcc(y=sound[i], sr=sr[i], n_mfcc=40)
    mfcc_length = mfccs.shape[1]
    if mfcc_length > max_length:
      max_length = mfcc_length

  return max_length

In [0]:
max_len = find_mfcc_max_length(sound, sr)
print(max_len)

3704


In [0]:
# Define feature extraction function: MFCC + demographic info
import librosa
max_len = 3704
def extract_feature(X, sample_rate, demographic_info):
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40)
    
    # Pad  mfcc till the end
    #pad_width_mfcc = max_len - mfccs.shape[1]
    #mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width_mfcc)), mode='constant')

    mean_mfcc = np.mean(mfccs.T,axis=0)
    std_mfcc = np.std(mfccs.T,axis=0)

    #feat = mfccs[0, :]
    #for i in range(39):
    #  feat = np.hstack((feat, mfccs[i+1, :]))
    #print(feat.shape)
    #return feat

    return np.hstack((mean_mfcc,std_mfcc))

In [0]:
# Extract features
featset = []
for i in range(len(sound)):
    a = extract_feature(sound[i], sr[i], demographic_data[i])
    featset.append(a)


In [13]:
print(len(featset))

920


In [0]:
new_patient_disease_labels = np.asarray(new_patient_disease_labels)
a = np.zeros(new_patient_disease_labels.shape[0])
for i in range(new_patient_disease_labels.shape[0]):
    for j in range(new_patient_disease_labels.shape[1]):
        if new_patient_disease_labels[i][j]==1:
            a[i]=j

# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(a)
oh_labels = to_categorical(i_labels) 

In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(featset, i_labels,test_size=0.3, random_state=42,stratify=i_labels)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [16]:
# SVM
Cs = [2**(-2),2**(-1), 1,2**(1),2**(2),2**(3),2**(4),2**(5),2**(6),2**(7),2**(8),2**(9),2**(10)]
gamma = [2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2**(0),2**(1),2**(2),2**(3)]


param_grid = {'C': Cs,  
                'gamma': gamma, 
                'kernel': ['rbf'],
                'decision_function_shape':['ov'],
                'class_weight': ['balanced']}  
grid1 = GridSearchCV(SVC(), param_grid,cv=3,n_jobs=-1, verbose = 3) 

# fitting the model for grid search 
grid1.fit(x_train, y_train)

# print best parameter after tuning 
print(grid1.best_params_) 
# print how our model looks after hyper-parameter tuning 
print(grid1.best_estimator_)

grid_predictions = grid1.predict(x_test) 

# print classification report 
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test,grid_predictions))

print(sklearn.metrics.confusion_matrix(y_test,grid_predictions))

Fitting 3 folds for each of 143 candidates, totalling 429 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    2.1s


{'C': 4, 'class_weight': 'balanced', 'decision_function_shape': 'ov', 'gamma': 0.0078125, 'kernel': 'rbf'}
SVC(C=4, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ov', degree=3, gamma=0.0078125, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.74      0.89      0.81        38
           1       0.98      0.95      0.97       238

    accuracy                           0.94       276
   macro avg       0.86      0.92      0.89       276
weighted avg       0.95      0.94      0.94       276

0.9420289855072463
[[ 34   4]
 [ 12 226]]


[Parallel(n_jobs=-1)]: Done 429 out of 429 | elapsed:   10.3s finished


In [17]:
# XgBoost
import xgboost as xgb

print('Training XGB Classifier from new features:')
xgb_model = xgb.XGBClassifier(max_depth=3,num_class=2, n_estimators=15, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='multi:softmax', eta=0.3, silent=0, subsample=0.8).fit(x_train, y_train)

xgb_pred = xgb_model.predict(x_test)
# print classification report 
print(classification_report(y_test, xgb_pred))
print(accuracy_score(y_test,xgb_pred))

Training XGB Classifier from new features:
              precision    recall  f1-score   support

           0       0.86      0.50      0.63        38
           1       0.93      0.99      0.96       238

    accuracy                           0.92       276
   macro avg       0.89      0.74      0.79       276
weighted avg       0.92      0.92      0.91       276

0.9202898550724637
